In [9]:
import requests
from urllib.parse import urlencode

In [64]:
class NCBI_Searcher:
    """Classe que implementa a utilização da API das databases da NCBI.
    """
    
    search_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi'
    meta_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi'
    
    def search(self, queryterms: list=None, db: str='pmc',
               retmode: str='json', retmax: int=20, retstart: int=0):
        """
        Realiza uma pesquisa NCBI.
        
        @param queryterms: list of lists. Terms within the same list are
            separated by an OR. Lists are separated by an AND
        @param db: base de dados a ser pesquisada. Se for mais de uma,
            pode ser separado por vírgula. As mais úteis serão:
            pmc, pubmed
        @param retmode: A forma de retorno. None será em XML.
        @@param retmax: A quantidade de artigos por pesquisa
        @param retstart: O 'offset' de início de retorno. Se setado
            para zero, irá trazer os resultados a partir do primeiro.
            Se setado para 10, irá trazer os resultados a partir do
            décimo.
            
        @return: uma lista de títulos e IDs no formato [(title, id)]
        """
        term = " AND ".join(["(%s)" % " OR ".join(orses)for orses in queryterms])
        payload = {"term":term, "db":db, "retmode":retmode,
                   "retmax": retmax, "retstart":retstart}
        url = "%s?%s" % (self.search_url, urlencode(payload))
        
        id_list = requests.get(url).json()['esearchresult']['idlist']
        
        result = self.get_article_metadata(*id_list)['result']
        
        return [(result[uid]['title'], uid) for uid in result['uids']]
    
    def get_article_metadata(self, *args, db: str='pmc', retmode: str='json'):
        """
        Retorna os metadados do(s) artigo(s).
        
        @param id: ID do artigo. O ID do artigo depende da base de dados.
            O ID de um artigo pode ser diferente entre a PMC e a PubMed
            por exemplo. Podem ser passados vários IDs ao mesmo tempo.
        @param db: base de dados a ser pesquisada. Se for mais de uma,
            pode ser separado por vírgula. As mais úteis serão:
            pmc, pubmed
        @param retmode: A forma de retorno. None será em XML.
        
        @return: o json cru retornado pela API.
        """
        id_list = ','.join([str(x) for x in args])
            
        payload = {"id":id_list, "db":db, "retmode":retmode}
        url = "%s?%s" % (self.meta_url, urlencode(payload))
        
        r = requests.get(url).json()
        
        return r
            
        
        
        
        

In [52]:
technology_queryterms = [
    'machine learning', 'deep learning', 'artificial intelligence', 
    'neural network', 'scoring system'
]

health_queryterms = [
    'coronary artery disease', 'chest pain', 'heart disease', 'MACE', 
    'Acute Cardiac Complications'
]

queryterms = [technology_queryterms, health_queryterms]

In [67]:
r = NCBI_Searcher().search(queryterms=queryterms)
[a[0] for a in r]

['Bivariate genome-wide association analyses identified genetic pleiotropic effects for bone mineral density and alcohol drinking in Caucasians',
 'KIF2A regulates the development of dentate granule cells and postnatal hippocampal wiring',
 'Understanding uptake of an intervention to accelerate antiretroviral therapy initiation in Uganda via qualitative inquiry',
 'Self-reported medication use validated through record linkage to national prescribing data',
 'A Reliable Grading System for Prediction of Chronic Subdural Hematoma Recurrence Requiring Reoperation After Initial Burr-Hole Surgery',
 'Altered DNA methylation indicates an oscillatory flow mediated epithelial-to-mesenchymal transition signature in ascending aorta of patients with bicuspid aortic valve',
 'Primary Endovascular Treatment of Acute Ischemic Stroke Using Stent Retrievers: Initial Egyptian Experience',
 'An fMRI study of the effects on normal language areas when acupuncturing the Tongli (HT5) and Xuanzhong (GB39) acu